# Temperature Tutorial

In this tutorial, we'll run through the basics of generating stream temperature time series predictions. We'll assume that we've already [extracted a collection of representative elementary watersheds (REWs)](network_extraction.ipynb), [paramterized and configured the REWs](parameterize_and_configure.ipynb), [generated hillslope discharge time series](hillslope_discharge.ipynb), and [routed discharge through the channel network](channel_routing.ipynb). 

In [1]:
from matplotlib import pyplot as plt
import matplotlib 
%matplotlib inline
import pandas as pd
import gdal
import os
import datetime
from datetime import timedelta
import numpy as np
import fiona
import shapely
from shapely import geometry
from os.path import dirname
import glob
import seaborn
import sys
import pickle
from functools import partial
import seaborn as sns
parent_dir = dirname(dirname(os.getcwd()))
sys.path.append(os.path.join(parent_dir,'StreamflowTempModel','lib'))
sys.path.append(os.path.join(parent_dir,'StreamflowTempModel','4_temperature'))
sys.path.append(os.path.join(parent_dir,'StreamflowTempModel','3_channel_routing'))
from temperature import SimpleTemperature, LagrangianSimpleTemperature
from channel import SimpleChannel
import zonal_stats as zs
import meteolib as meteo
import evaplib as evap

In [2]:
try:
    basins = glob.glob(os.path.join(parent_dir,'raw_data','basins_poly','*.shp'))[0]
except RuntimeError:
    print 'Cannot find basins shapefile. Please make sure basins shapefile is located in \n the model directory under /raw_data/basins_poly'

We first need to gain access to all of the parameterization, configuration, hillslope discharge, and routed channel flow data that we've already generated. 

In [3]:
rew_config = pickle.load( open( os.path.join(parent_dir,'model_data','rew_config.p'), "rb" ) )
climate_group_forcing = pickle.load( open( os.path.join(parent_dir,'model_data','climate_group_forcing.p'), "rb" ) )
model_config = pickle.load( open( os.path.join(parent_dir, 'model_data', 'model_config.p'), 'rb'))
temperature_params = pickle.load( open( os.path.join(parent_dir, 'model_data', 'temperature_params.p'), 'rb'))
hill_groups = pickle.load( open( os.path.join(parent_dir,'model_data','solved_hillslope_discharge.p'), "rb" ) )
solved_channel_routing = pickle.load( open( os.path.join(parent_dir,'model_data','solved_channel_routing.p'), "rb" ) )
channel_params = pickle.load( open( os.path.join(parent_dir,'model_data','channel_params.p'), "rb" ))
radiation = pickle.load( open(os.path.join(parent_dir, 'raw_data', 'radiation', 'radiation.p'),'rb') )
ta_ea = pickle.load( open(os.path.join(parent_dir, 'raw_data', 'ta_ea', 'ta_ea.p'),'rb') )

#start/stop dates for running model  
#spinup date is the date after start_date for which we assume model is finished spinning up         
start_date = model_config['start_date']
stop_date = model_config['stop_date']
spinup_date = model_config['spinup_date']
Tmax = model_config['Tmax']
dt = model_config['dt_temperature']
resample_freq_channel = model_config['resample_freq_channel']
resample_freq_hillslope = model_config['resample_freq_hillslope']
resample_freq_temperature = model_config['resample_freq_temperature']
t = np.linspace(0,Tmax,np.ceil(Tmax/dt)+1)
timestamps_hillslope = pd.date_range(start_date, stop_date, freq=resample_freq_hillslope)
timestamps_channel = pd.date_range(start_date, stop_date, freq=resample_freq_channel)
timestamps_temperature = pd.date_range(start_date, stop_date, freq=resample_freq_temperature)





With these basic data now loaded, we can instantiate both the channel network and the temperature networks. 

In [4]:
channel_network = {}
for rew_id in rew_config.keys(): 
    args = rew_config[rew_id].copy()
    args.update(channel_params[rew_id])
    channel_network[rew_id] = args['model'](rew_id=rew_id, **args)

temperature_network = {}
for rew_id in rew_config.keys(): 
    args = rew_config[rew_id].copy()
    args.update(temperature_params[rew_id])
    temperature_network[rew_id] = args['model'](rew_id=rew_id, **args)

We simulate the temperature time series beginning with the headwaters, i.e. those with Shreve index 1. These temperature time series are then used as boundary conditions for streams with higher Shreve. 

In [5]:
#Find REW with largest total contributing area
outlet_id = rew_config.keys()[0]
area_max = rew_config[outlet_id]['upstream_area']
for rew_id in rew_config.keys():
    if rew_config[rew_id]['upstream_area']>area_max:
        outlet_id = rew_id

rew_ids = rew_config.keys()
shreves = [rew_config[rew_id]['shreve'] for rew_id in rew_ids]
rewQueue = [rew_id for (shreve,rew_id) in sorted(zip(shreves,rew_ids))]
network_temps = {}
network_airtemp = {}
print "Solve REWs in this order:"
print(rewQueue)

for rew_id in rewQueue:
    
    print 'Working on REW ' + str(rew_id)
    shreve  = rew_config[rew_id]['shreve']
    group_id = rew_config[rew_id]['group']
    climate_group_id = group_id[1]
    rew_df = climate_group_forcing[climate_group_id]
    width = channel_network[rew_id].width
    length = channel_network[rew_id].length
    
    Lin = np.array(radiation[rew_id]['Lin'][start_date:stop_date].resample(resample_freq_temperature).ffill())
    Sin = np.array(radiation[rew_id]['Sin'][start_date:stop_date].resample(resample_freq_temperature).ffill())
    
    temp_ea = ta_ea[rew_id].resample(resample_freq_temperature).interpolate()
    ppt_daily = climate_group_forcing[climate_group_id][start_date:stop_date].ppt
    ppt = np.array(climate_group_forcing[climate_group_id][start_date:stop_date].ppt.resample(resample_freq_temperature).ffill())
    ta = np.array(temp_ea['ta'][start_date:stop_date])
    ea = np.array(temp_ea['ea'][start_date:stop_date])
    hillslope_discharge = pd.DataFrame({'discharge':hill_groups[group_id]['discharge']}, index=hill_groups[group_id].index)
    hillslope_overlandFlow = pd.DataFrame({'overlandFlow':hill_groups[group_id]['overlandFlow']}, index=hill_groups[group_id].index)
    
    hillslope_volumetric_overlandFlow = np.array(hillslope_overlandFlow[start_date:stop_date].overlandFlow.resample(resample_freq_temperature).ffill())*rew_config[rew_id]['area_sqcm']
    hillslope_volumetric_discharge = np.array(hillslope_discharge[start_date:stop_date].discharge.resample(resample_freq_temperature).ffill())*rew_config[rew_id]['area_sqcm']
    hillslope_volumetric_discharge_daily = hillslope_discharge[start_date:stop_date].discharge*rew_config[rew_id]['area_sqcm']
    
    volumetric_discharge = np.array(solved_channel_routing[rew_id][start_date:stop_date].volumetric_discharge.resample(resample_freq_temperature).ffill())
    volumetric_discharge_daily = solved_channel_routing[rew_id][start_date:stop_date].volumetric_discharge
    temp = np.zeros(np.shape(t))
    
    #get upstream discharges, upstream temperatures
    vol_1_daily = 0
    vol_2_daily = 0
    if shreve == 1:
        vol_1 = np.zeros(np.shape(t))
        vol_2 = np.zeros(np.shape(t))

        temp_1 = np.zeros(np.shape(t))
        temp_2 = np.zeros(np.shape(t))
    else:
        upstream_1 = rew_config[rew_id]['prev_str01']
        upstream_2 = rew_config[rew_id]['prev_str02']

        vol_1 = np.array(solved_channel_routing[upstream_1][start_date:stop_date].volumetric_discharge.resample(resample_freq_temperature).ffill())
        vol_2 = np.array(solved_channel_routing[upstream_2][start_date:stop_date].volumetric_discharge.resample(resample_freq_temperature).ffill())

        vol_1_daily = solved_channel_routing[upstream_1][start_date:stop_date].volumetric_discharge[0]
        vol_2_daily = solved_channel_routing[upstream_2][start_date:stop_date].volumetric_discharge[0]
        
        temp_1 = np.array(network_temps[upstream_1].temperature)
        temp_2 = np.array(network_temps[upstream_2].temperature)

    # Now get volumes in channel link. 
    volume = np.array(solved_channel_routing[rew_id][start_date:stop_date].volumes.resample(resample_freq_temperature).interpolate())
    start_temp_model = int(1/dt*(len(pd.date_range(start_date,spinup_date))-365))
    for i in range(len(t)):
        if i<start_temp_model:
            temp[i] = temperature_network[rew_id].temperature
        else:
            varyArgs = ['vol_1','temp_1','vol_2','temp_2','hillslope_volumetric_discharge', 'hillslope_volumetric_overlandFlow', 'volumetric_discharge', 'volume', 'ta', 'Lin', 'Sin', 'ppt', 'ea']
            constArgs = ['width','length']

            tempArgs = {}
            for arg in varyArgs: tempArgs[arg] = globals()[arg][i]
            for arg in constArgs: tempArgs[arg] = globals()[arg]

            temp[i]=temperature_network[rew_id].temperature
            temperature_network[rew_id].update(dt, **tempArgs)
            
    
            
    depth = volume/(length*width)
    network_airtemp[rew_id] = pd.DataFrame({'ta':ta, 'ppt':ppt, 'volume':volume, 'depth':depth, 'Sin':Sin}, index=timestamps_temperature)
    network_temps[rew_id] = pd.DataFrame({'temperature':temp},index=timestamps_temperature)

Solve REWs in this order:
[1, 2, 3]
Working on REW 1
Working on REW 2


KeyboardInterrupt: 

In [6]:
sns.set(font_scale=4)
plt.figure(figsize=(45,20))
temp_data = pickle.load( open(os.path.join(parent_dir,'calibration_data','elder_temperature.p'), "rb" ))

plt.plot(network_temps[3],alpha=.3, linewidth=4)
plt.plot(temp_data.temperature)
plt.plot(network_airtemp[1].ta,alpha=0.5)
plt.plot(-network_airtemp[1].ppt)
# plt.plot((network_airtemp[1].Sin-10)/10)
# plt.plot(network_airtemp[1].depth)

plt.axvspan(start_date, spinup_date, color='r', alpha=0.1, lw=0)
plt.axvspan(spinup_date, stop_date, color='g', alpha=0.1, lw=0)
plt.xlim(['10-2012','10-2014'])

# plt.ylim([-3,20])

KeyError: 3

In [ ]:
# Mass balance from final iteration
cumInDis = np.cumsum(vol_1 + vol_2 + hillslope_volumetric_discharge)*dt
cumPpt = np.cumsum(ppt)*dt*length*width
cumOutDis = np.cumsum(volumetric_discharge)*dt
S0 = volume[0]
totalIn = np.array(cumInDis + cumPpt)
S = np.array(volume)
totalOut = np.array(cumOutDis)
balance = (totalIn[:-1] - totalOut[:-1] + S0)/S[1:]
plt.plot(balance)
print np.max(balance)
print np.min(balance)

In [ ]:
# # check balance of an individual channel 
# network_volumetric_discharges = pickle.load( open( os.path.join(parent_dir,'model_data','solved_channel_routing.p'), "rb" ) )

# rew=3
# stream = network_volumetric_discharges[rew].volumes
# length = channel_network[rew].length
# width = channel_network[rew].width
# area = length*width
# group = rew_config[rew]['group']
# climate_group_id = group_id[1]
# ppt = climate_group_forcing[climate_group_id][start_date:stop_date].ppt*area 
# hillslope_discharge = pd.DataFrame({'discharge':hill_groups[group_id]['discharge']}, index=hill_groups[group_id].index)
# hillslope_overlandFlow = pd.DataFrame({'overlandFlow':hill_groups[group_id]['overlandFlow']}, index=hill_groups[group_id].index)
# hillslope_discharge['discharge'] = hillslope_discharge['discharge'] + hillslope_overlandFlow['overlandFlow']
# hillslope_volumetric_discharge = hillslope_discharge[start_date:stop_date].discharge*rew_config[rew]['area_sqcm']


# outDis = network_volumetric_discharges[rew].volumetric_discharge
# cumOutDis = outDis.cumsum() 

# upstream_1 = rew_config[rew]['prev_str01']
# upstream_2 = rew_config[rew]['prev_str02']
# try:
#     vol_1 = network_volumetric_discharges[upstream_1].volumetric_discharge#.resample(resample_freq_channel).ffill()
#     vol_2 = network_volumetric_discharges[upstream_2].volumetric_discharge#.resample(resample_freq_channel).ffill()
#     inDis = vol_1 + vol_2 + hillslope_volumetric_discharge
# except:
#     inDis = hillslope_volumetric_discharge
        

# cumInDis = inDis.cumsum()

# cumPpt = ppt.cumsum()


# S0 = stream[0] + cumOutDis[0] - cumPpt[0] - cumInDis[0]
# totalIn = cumPpt + cumInDis
# S =  stream
# totalOut = cumOutDis
# balance = (totalIn - totalOut + S0)/S
# plt.plot(balance[0:-2])
# plt.ylim([0,1.1])
# print(np.max(balance[0:-2]))
# print(np.min(balance[0:-2]))

In [ ]:

# rew = 3
# group_id = rew_config[rew]['group']
# climate_group_id = group_id[1]
# network_volumetric_discharges = pickle.load( open( os.path.join(parent_dir,'model_data','solved_channel_routing.p'), "rb" ) )

# # volume in outlet rew stream
# stream = network_volumetric_discharges[rew].volumes.resample(resample_freq_temperature).interpolate()
# width = network_volumetric_discharges[rew].width[0]
# length = network_volumetric_discharges[rew].length[0]

# # volumetric discharge leaving rew stream
# volumetric_discharge = network_volumetric_discharges[rew].volumetric_discharge.resample(resample_freq_temperature).bfill()

# # precip on channel
# area = width*length
# ppt = climate_group_forcing[climate_group_id][start_date:stop_date].ppt.resample(resample_freq_temperature).bfill()*area 

# # incoming hillslope discharge 
# hillslope_discharge = pd.DataFrame({'discharge':hill_groups[group_id]['discharge']}, index=hill_groups[group_id].index)
# hillslope_volumetric_discharge = hillslope_discharge[start_date:stop_date].discharge.resample(resample_freq_temperature).bfill()*rew_config[rew]['area_sqcm']

# # volumetric discharges from upstream
# upstream_1 = rew_config[rew]['prev_str01']
# upstream_2 = rew_config[rew]['prev_str02']            
# vol_1 = network_volumetric_discharges[upstream_1].volumetric_discharge.resample(resample_freq_temperature).bfill()
# vol_2 = network_volumetric_discharges[upstream_2].volumetric_discharge.resample(resample_freq_temperature).bfill()
                                          
# cumPpt = np.cumsum(ppt)*dt
# cumInDis = np.cumsum(vol_1 + vol_2 + hillslope_volumetric_discharge)*dt
# cumOutDis = np.cumsum(volumetric_discharge)*dt

# S0 = stream[0] - cumInDis[0] + cumOutDis[0] - cumPpt[0]
# S = stream
# totalIn = cumPpt + cumInDis
# totalOut = cumOutDis
# balance = (totalIn - totalOut + S0)/S
# print(np.max(balance[0:-int(1/dt)]))
# print(np.min(balance[0:-int(1/dt)]))
# plt.plot(balance[0:-int(1/dt)])

In [ ]:
sns.set(font_scale=1)
plt.plot(network_temps[1].temperature.loc['10-5-2012':'10-15-2012'])

In [ ]:
network_temps[1].temperature

In [ ]:
hillslope_discharge[start_date:stop_date].index.month>8